## Final Activity 1

Objectives:
- Develop regression models for a given dataset and interpret the regression models.
- Evaluate the regression models based on the regression metrics.

#### Simple Linear Regression (SLR)

Predict how much discount should be given considering any damage in the house. The dataset has a column for number of damages and another column for the discount that can be given (around USD 1 for every damage discovered)

#### Multiple Linear Regression (MLR)

Predict the possible price of a house by correlating the price column with ONLY the following attributes/columns in the dataset:
- size (in square meters)
- number of bedrooms
- number of bathrooms
- number of extra rooms
- presence of a garage (0 - no, 1- yes)
- presence of a garden (0 - no, 1- yes)
- if the house is in a subdivision (0 - no, 1- yes)
- if the house is located in a city (0 - no, 1- yes)
- if the house is solar powered (0 - no, 1- yes)

In [4]:
# Import Dataset
import pandas as pd

raw = pd.read_csv('data/housePriceData.csv')

raw

,size,bedrooms,bathrooms,extraRooms,garage,garden,inSubdivision,inCity,solarPowered,price,damages,discount
0,164,2,2,3,0,0,0,1,1,43800,77,79.775152
1,84,2,4,2,0,1,1,0,1,37550,21,23.177279
2,190,2,4,2,1,0,0,1,0,49500,22,25.609262
3,75,2,4,1,0,1,1,1,1,50075,20,17.857388
4,148,1,2,2,1,1,0,0,1,52400,36,41.849864
...,...,...,...,...,...,...,...,...,...,...,...,...
295,234,2,5,1,0,0,0,0,0,27600,71,68.545888
296,73,1,3,2,1,1,0,0,0,46825,46,47.334876
297,32,3,5,3,1,0,0,1,1,55750,55,54.090637
298,118,3,1,3,0,0,1,0,1,28400,62,63.297171
